In [1]:
!pip install langchain langchain-openai langchain-community sentence-transformers chromadb


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# openai api key
import os
os.environ['OPENAI_API_KEY'] = "XXXXXXXX"

In [3]:
# rag search
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.vectorstores import Chroma

In [4]:
# retriever
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
texts = ["Harrison worked at Kensho", "Bears like to eat honey"]
retriever = Chroma.from_texts(texts, embedding=embeddings).as_retriever()

C:\Users\Tong\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Tong\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
# chain
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

In [6]:
# invoke
print(chain.invoke("where did harrison work?"))

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


Harrison worked at Kensho.
